### 25단계 계산 그래프 시각화(1)

- Graphviz를 사용하여 계산 그래프를 시각화

#### 25.1 Graphviz 설치하기

[다운로드](http://graphviz.gitlab.io/download/)

- 버전 확인
```
$ dot -V
```

- 파일 변환
```
$ dot sample.dot -T png -o sample.png
```

#### 25.2 DOT 언어로 그래프 작성하기

- 편집기에서 테스트를 입력
```
digraph g{
x
y
}
```
- 노드는 줄바꿈으로 구분
- sample.dot 파일로 저장
- 파일 변환 명령어 실행
```
$ dot sample.dot -T png -o sample.png
```

![](./image/25-1.png)

#### 25.3 노드에 속성 지정하기

- 노드에 '색'과 '모양'을 지정할 수 있음
```
digraph g{
1 [label='x', color=orange, style=filled] # 오랜지색 채워진 타원형
2 [label='y', color=orange, style=filled]
3 [label='Exp', color=lightblue, style=filled, shape=box] # 파란색 사각형
}
```
- 노드 ID는 0이상의 정수, 중복 x

#### 25.4 노드 연결하기

- ID를 '->'로 연결
```
digraph g{
1 [label='x', color=orange, style=filled] # 오랜지색 채워진 타원형
2 [label='y', color=orange, style=filled]
3 [label='Exp', color=lightblue, style=filled, shape=box] # 파란색 사각형
1 -> 3
3 -> 2
}
```

### 26단계 계산 그래프 시각화(2)

- DeZero에서 실행한 계산을 DOT 언어로 변환하는 기능을 구현

#### 26.1 시각화 코드 예

- 시각화 함수 get_dot_graph를 dezero/utils.py에 구현

#### 26.2 계산 그래프에서 DOT 언어로 변환하기

In [16]:
def _dot_var(v, verbose=False):
    dot_var = '{} [label="{}", color=orange, style=filled]\n'

    name = '' if v.name is None else v.name
    if verbose and v.data is not None:
        if v.name is not None:
            name += ': '
        name += str(v.shape) + ' ' + str(v.dtype)

    return dot_var.format(id(v), name)

In [18]:
# 사용 예
x = Variable(np.random.randn(2, 3))
x.name = 'x'
print(_dot_var(x))
print(_dot_var(x, verbose=True))

1967699485312 [label="x", color=orange, style=filled]

1967699485312 [label="x: (2, 3) float64", color=orange, style=filled]



- DeZero 함수를 DOT 언어로 변환하는 편의 함수

In [20]:
def _dot_func(f):
    # for function
    dot_func = '{} [label="{}", color=lightblue, style=filled, shape=box]\n'
    ret = dot_func.format(id(f), f.__class__.__name__)

    # for edge
    dot_edge = '{} -> {}\n'
    for x in f.inputs:
        ret += dot_edge.format(id(x), id(f))
    for y in f.outputs:  # y is weakref
        ret += dot_edge.format(id(f), id(y()))
    return ret

In [21]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
y = x0 + x1
txt = _dot_func(y.creator)
print(txt)

1967699042016 [label="Add", color=lightblue, style=filled, shape=box]
1967699041200 -> 1967699042016
1967699039904 -> 1967699042016
1967699042016 -> 1967699039184



- get_dot_graph 함수 구현

In [24]:
def get_dot_graph(output, verbose=True):
    txt = ''
    funcs = []
    seen_set = set()

    def add_func(f):
        if f not in seen_set:
            funcs.append(f)
            # funcs.sort(key=lambda x: x.generation)
            seen_set.add(f)

    add_func(output.creator)
    txt += _dot_var(output, verbose)

    while funcs:
        func = funcs.pop()
        txt += _dot_func(func)
        for x in func.inputs:
            txt += _dot_var(x, verbose)

            if x.creator is not None:
                add_func(x.creator)

    return 'digraph g {\n' + txt + '}'

- Variable 클래스의 backward 메서드와 거의 같음
- backward 메서드는 미분값을 전파, 미분 대신 DOT 언어로 기술한 문자열을 txt에 추가

#### 26.3 이미지 변환까지 한 번에

In [ ]:
def plot_dot_graph(output, verbose=True, to_file='graph.png'):
    dot_graph = get_dot_graph(output, verbose)

    # 1 dot 데이터를 파일에 저장
    tmp_dir = os.path.join(os.path.expanduser('~'), '.dezero')
f plot_dot_graph(output, verbose=True, to_file='graph.png'):
    dot_graph = get_dot_graph(output, verbose)

    # 1 dot 데이터를 파일에 저장
    tmp_dir = os.path.join(os.path.expanduser('~'), '.dezero')
    if not os.path.exists(tmp_dir): # ~/.dezero 디렉터리가 없다면 새로 생성
        os.mkdir(tmp_dir)
    graph_path = os.path.join(tmp_dir, 'tmp_graph.dot')

    with open(graph_path, 'w') as f:
        f.write(dot_graph)

    # 2 dot 명령 호출
    extension = os.path.splitext(to_file)[1][1:]  # Extension(e.g. png, pdf)
    cmd = 'dot {} -T {} -o {}'.format(graph_path, extension, to_file)
    subprocess.run(cmd, shell=True)

    # Return the image as a Jupyter Image object, to be displayed in-line.
    try:
        from IPython import display
        return display.Image(filename=to_file)
    except:
        pass

- 1: 방금 구현한 get_dot_graph 함수를 호출하여 계산 그래프를 DOT 언어로 변환하고 파일에 저장
- 2: 저장한 파일 이름을 지정하여 dot 명령을 호출, 이때 plot_dot_graph 함수의 인수인 to_file에 저장할 이미지 파일의 이름을 지정
- 파이썬에서 외부 프로그램을 호출하기 위해 subprocess.run 함수를 사용

In [33]:
'''
Need the dot binary from the graphviz package (www.graphviz.org).
'''
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import Variable
from dezero.utils import plot_dot_graph


def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()

x.name = 'x'
y.name = 'y'
z.name = 'z'
plot_dot_graph(z, verbose=False, to_file='goldstein.png')

### 27단계 테일러 급수 미분

- sin 함수를 DeZero로 구현, 그 미분을 테일러 급수를 이용해 계산

#### 27.1 sin 함수 구현

In [35]:
class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = gy * np.cos(x)
        return gx


def sin(x):
    return Sin()(x)

In [36]:
x = Variable(np.array(np.pi / 4))
y = sin(x)
y.backward()
print('--- original sin ---')
print(y.data)
print(x.grad)

--- original sin ---
0.7071067811865476
0.7071067811865476


#### 27.2 테일러 급수 이론

- 테일러 급수란 어떤 함수를 다항식으로 근사하는 방법

![](./image/27.1.png)

#### 27.3 테일러 급수 구현

- 계승 계산은 파이썬의 math 모듈에 있는 math.factorial 함수를 사용

In [39]:
import math
def my_sin(x, threshold=0.0001):
    y = 0
    for i in range(100000):
        c = (-1) ** i / math.factorial(2 * i + 1)
        t = c * x ** (2 * i + 1)
        y = y + t
        if abs(t.data) < threshold:
            break
    return y

In [40]:
x = Variable(np.array(np.pi / 4))
y = my_sin(x)  # , threshold=1e-150)
y.backward()
print('--- approximate sin ---')
print(y.data)
print(x.grad)

--- approximate sin ---
0.7071064695751781
0.7071032148228457


### 29단계 뉴턴 방법으로 푸는 최적화(수동계산)